# Overview
The goal of this notebook is to compare state and sector energy by kWh per capita consumption.

Electric Sales - Sourced from EIA and kWh actually consumed
Per capita - Sourced from EIA and based on customer electricity accounts 

Sectors include - Residential (RES), Commericial (COM), Transporation (TRA), Industrial (IND), and Other (OTH)


Still To Do:
* Check kWh per capita calculation for states AR, AK, and AZ.  It's odd that they all start with the letter A.
* Place plots into slide deck

Notebooks sources:
* [Electricity Sales Ingestion]("https://colab.research.google.com/drive/1MtcvMAsxUsEa0wHABkpf-sWS9h8XTJw4#scrollTo=kg5tIuCH1h97")
* [Electric Account Data]("https://colab.research.google.com/drive/1KTLQhMf1NVf6Y74d9tVHEQWEm8LPEqLl#scrollTo=JQ7wD-d6mm4N")

In [ ]:

from google.colab import drive

drive.mount('./drive/')

%cd drive/My \ Drive/Data606_Energy


Drive already mounted at ./drive/; to attempt to forcibly remount, call drive.mount("./drive/", force_remount=True).
[Errno 2] No such file or directory: 'drive/My  Drive/Data606_Energy'
/content


In [ ]:
import sys
import pandas as pd
import glob
from tqdm.autonotebook import tqdm
sys.path.append("helpers/")

from energygrid import EGRID

from helper_functions import combine_like_files,clean_electricity


import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  after removing the cwd from sys.path.


In [ ]:
sys.path.append("helpers/")
energy_grid = EGRID()
energy_grid.get_states()

# Retail Electricty Sales in kWh - https://www.eia.gov/opendata/qb.php?category=38&sdid=ELEC.SALES.AL-ALL.M

## Definition - Retail sales (electric):  Sales made directly to the customer that consumes the energy product.

In [ ]:
#Read file
df_total_electric_retail_sales = combine_like_files("data/electricity_retail_sales/*/*_retail_sales.json",pd.read_json)
df_total_electric_retail_sales.sample(10)

,series_id,name,units,f,description,copyright,source,iso3166,geography,start,end,updated,data,sector
65539,ELEC.SALES.NH-ALL.M,Retail sales of electricity : New Hampshire : ...,million kilowatthours,M,All end-use sectors that consume electricity; ...,None,"EIA, U.S. Energy Information Administration",USA-NH,USA-NH,200101,202107,2021-09-23T14:54:35-0400,"[201407, 975.77949]",ALL
20982,ELEC.SALES.NY-COM.M,Retail sales of electricity : New York : comme...,million kilowatthours,M,Electricity sold to commercial customers; Elec...,None,"EIA, U.S. Energy Information Administration",USA-NY,USA-NY,200101,202107,2021-09-23T14:54:35-0400,"[200201, 5023.24502]",COM
35118,ELEC.SALES.SC-IND.M,Retail sales of electricity : South Carolina :...,million kilowatthours,M,Electricity sold to industrial customers; Elec...,None,"EIA, U.S. Energy Information Administration",USA-SC,USA-SC,200101,202107,2021-09-23T14:54:35-0400,"[201711, 2191.63693]",IND
26168,ELEC.SALES.CA-IND.M,Retail sales of electricity : California : ind...,million kilowatthours,M,Electricity sold to industrial customers; Elec...,None,"EIA, U.S. Energy Information Administration",USA-CA,USA-CA,200101,202107,2021-09-23T14:54:35-0400,"[200202, 3718.73132]",IND
26070,ELEC.SALES.CA-IND.M,Retail sales of electricity : California : ind...,million kilowatthours,M,Electricity sold to industrial customers; Elec...,None,"EIA, U.S. Energy Information Administration",USA-CA,USA-CA,200101,202107,2021-09-23T14:54:35-0400,"[201004, 3959.92863]",IND
15925,ELEC.SALES.ID-COM.M,Retail sales of electricity : Idaho : commerci...,million kilowatthours,M,Electricity sold to commercial customers; Elec...,None,"EIA, U.S. Energy Information Administration",USA-ID,USA-ID,200101,202107,2021-09-23T14:54:35-0400,"[201110, 464.28351]",COM
8599,ELEC.SALES.ND-RES.M,Retail sales of electricity : North Dakota : r...,million kilowatthours,M,Electricity sold to residential customers; Ele...,None,"EIA, U.S. Energy Information Administration",USA-ND,USA-ND,200101,202107,2021-09-23T14:54:35-0400,"[200410, 275.83323]",RES
48205,ELEC.SALES.TN-OTH.M,Retail sales of electricity : Tennessee : othe...,million kilowatthours,M,Electricity sold to the other category (rolled...,None,"EIA, U.S. Energy Information Administration",USA-TN,USA-TN,200101,202107,2021-09-23T14:54:35-0400,"[201803, 0]",OTH
7310,ELEC.SALES.NH-RES.M,Retail sales of electricity : New Hampshire : ...,million kilowatthours,M,Electricity sold to residential customers; Ele...,None,"EIA, U.S. Energy Information Administration",USA-NH,USA-NH,200101,202107,2021-09-23T14:54:35-0400,"[200904, 346.96368]",RES
123,ELEC.SALES.AL-RES.M,Retail sales of electricity : Alabama : reside...,million kilowatthours,M,Electricity sold to residential customers; Ele...,None,"EIA, U.S. Energy Information Administration",USA-AL,USA-AL,200101,202107,2021-09-23T14:54:35-0400,"[201104, 2038.14668]",RES


In [ ]:
df_total_electric_sales_clean = clean_electricity(df_total_electric_retail_sales,"kWh_million")

df_total_electric_sales_clean["kWh"] = df_total_electric_sales_clean["kWh_million"] * 1000000

In [ ]:
df_total_electric_sales_clean

In [ ]:
query = df_total_electric_sales_clean.query('sector=="ALL"')

all_without_percapita_context = px.scatter(query,x="date",y="kWh",color="state",title="Electricity Sales - Without Per Capita Context")
all_without_percapita_context

# Per Capita

In [ ]:
df_all_electric_accounts = combine_like_files("data/percapita/electric_accounts/json/*/*_electric_accounts.json",pd.read_json)

In [ ]:
df_all_electric_accounts = clean_electricity(df_all_electric_accounts,"electric_accounts")

In [ ]:
df_all_electric_accounts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38305 entries, 0 to 38304
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   series_id          38305 non-null  object        
 1   name               38305 non-null  object        
 2   f                  38305 non-null  object        
 3   description        38305 non-null  object        
 4   source             38305 non-null  object        
 5   updated            38305 non-null  object        
 6   sector             38305 non-null  object        
 7   country            38305 non-null  object        
 8   state              38305 non-null  object        
 9   electric_accounts  38019 non-null  float64       
 10  date               38305 non-null  datetime64[ns]
 11  year               38305 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(9)
memory usage: 3.5+ MB


In [ ]:
df_all_electric_accounts.sample(5)

,series_id,name,f,description,source,updated,sector,country,state,electric_accounts,date,year
12438,ELEC.CUSTOMERS.MO-COM.M,Number of customer accounts : Missouri : comme...,M,Electricity sold to commercial customers; Elec...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,COM,USA,MO,380706.0,2017-05-01,2017
24597,ELEC.CUSTOMERS.WV-IND.M,Number of customer accounts : West Virginia : ...,M,Electricity sold to industrial customers; Elec...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,IND,USA,WV,12173.0,2009-04-01,2009
21982,ELEC.CUSTOMERS.NY-IND.M,Number of customer accounts : New York : indus...,M,Electricity sold to industrial customers; Elec...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,IND,USA,NY,8407.0,2009-11-01,2009
18451,ELEC.CUSTOMERS.ID-IND.M,Number of customer accounts : Idaho : industri...,M,Electricity sold to industrial customers; Elec...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,IND,USA,ID,28224.0,2018-11-01,2018
8918,ELEC.CUSTOMERS.AR-COM.M,Number of customer accounts : Arkansas : comme...,M,Electricity sold to commercial customers; Elec...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,COM,USA,AR,180642.0,2011-11-01,2011


# Per Capita Electricity Sales

In [ ]:
#Merge datasets to get state population
df_electric_population = df_total_electric_sales_clean.merge(df_all_electric_accounts[["state","date","year","sector","electric_accounts"]],on=["state","date","year","sector"],how="left")

df_electric_population.sample(5)

,series_id,name,f,description,source,updated,sector,country,state,kWh_million,date,year,kWh,electric_accounts
8654,ELEC.SALES.OH-RES.M,Retail sales of electricity : Ohio : residenti...,M,Electricity sold to residential customers; Ele...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,RES,USA,OH,3486.46126,2020-10-01,2020,3.486461e+09,5068876.0
54391,ELEC.SALES.MO-TRA.M,Retail sales of electricity : Missouri : trans...,M,Electricity sold to transportation customers; ...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,TRA,USA,MO,1.75765,2017-04-01,2017,1.757650e+06,2.0
7294,ELEC.SALES.NH-RES.M,Retail sales of electricity : New Hampshire : ...,M,Electricity sold to residential customers; Ele...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,RES,USA,NH,437.21029,2010-08-01,2010,4.372103e+08,599495.0
12747,ELEC.SALES.AL-COM.M,Retail sales of electricity : Alabama : commer...,M,Electricity sold to commercial customers; Elec...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,COM,USA,AL,1801.24152,2009-01-01,2009,1.801242e+09,359702.0
54791,ELEC.SALES.NV-TRA.M,Retail sales of electricity : Nevada : transpo...,M,Electricity sold to transportation customers; ...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,TRA,USA,NV,0.00000,2004-07-01,2004,0.000000e+00,NaN


# Corrected Dataframe here - used to be df_all_electric_accounts

In [ ]:
#Calculate kWh per capita
df_electric_population["kWh_percapita"] = df_electric_population["kWh"] / df_electric_population["electric_accounts"]

In [ ]:
df_electric_population.query('sector == "ALL"')

,series_id,name,f,description,source,updated,sector,country,state,kWh_million,date,year,kWh,electric_accounts,kWh_percapita
58292,ELEC.SALES.AL-ALL.M,Retail sales of electricity : Alabama : all se...,M,All end-use sectors that consume electricity; ...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,ALL,USA,AL,8134.79498,2021-07-01,2021,8.134795e+09,2688806.0,3025.430239
58293,ELEC.SALES.AL-ALL.M,Retail sales of electricity : Alabama : all se...,M,All end-use sectors that consume electricity; ...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,ALL,USA,AL,7509.01996,2021-06-01,2021,7.509020e+09,2685634.0,2795.995270
58294,ELEC.SALES.AL-ALL.M,Retail sales of electricity : Alabama : all se...,M,All end-use sectors that consume electricity; ...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,ALL,USA,AL,6591.84032,2021-05-01,2021,6.591840e+09,2676816.0,2462.567588
58295,ELEC.SALES.AL-ALL.M,Retail sales of electricity : Alabama : all se...,M,All end-use sectors that consume electricity; ...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,ALL,USA,AL,6161.72519,2021-04-01,2021,6.161725e+09,2673895.0,2304.400580
58296,ELEC.SALES.AL-ALL.M,Retail sales of electricity : Alabama : all se...,M,All end-use sectors that consume electricity; ...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,ALL,USA,AL,6639.53506,2021-03-01,2021,6.639535e+09,2674292.0,2482.726292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70884,ELEC.SALES.WV-ALL.M,Retail sales of electricity : West Virginia : ...,M,All end-use sectors that consume electricity; ...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,ALL,USA,WV,2108.17448,2001-05-01,2001,2.108174e+09,NaN,NaN
70885,ELEC.SALES.WV-ALL.M,Retail sales of electricity : West Virginia : ...,M,All end-use sectors that consume electricity; ...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,ALL,USA,WV,2249.49921,2001-04-01,2001,2.249499e+09,NaN,NaN
70886,ELEC.SALES.WV-ALL.M,Retail sales of electricity : West Virginia : ...,M,All end-use sectors that consume electricity; ...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,ALL,USA,WV,2405.07420,2001-03-01,2001,2.405074e+09,NaN,NaN
70887,ELEC.SALES.WV-ALL.M,Retail sales of electricity : West Virginia : ...,M,All end-use sectors that consume electricity; ...,"EIA, U.S. Energy Information Administration",2021-09-23T14:54:35-0400,ALL,USA,WV,2411.13253,2001-02-01,2001,2.411133e+09,NaN,NaN


# Per Capita Calculation

$$ \text{kWh_percapita} = \frac{\text{kWh}}{\text{electric_accounts}}$$

# Check among all states starting with 2008

In [ ]:
df_electric_population[["date","state","sector","kWh_percapita"]].query('kWh_percapita == "inf"').groupby(["state","sector"]).count()

,,date,kWh_percapita
state,sector,,
LA,TRA,11,11
NC,TRA,11,11
UT,TRA,4,4


# *Come back AR, AK look too high

In [ ]:
"""
comp_df_state_date_sector = df_electricity_sales_sector_clean.groupby(["state","date"]).sum().drop(columns="year")
comp_df_state_date_all = df_total_electric_sales_clean.groupby(["state","date"]).sum().drop(columns="year")
comp_df_state_date_sector["difference"] = comp_df_state_date_sector["kWh"] - comp_df_state_date_all["kWh"]
"""

'\ncomp_df_state_date_sector = df_electricity_sales_sector_clean.groupby(["state","date"]).sum().drop(columns="year")\ncomp_df_state_date_all = df_total_electric_sales_clean.groupby(["state","date"]).sum().drop(columns="year")\ncomp_df_state_date_sector["difference"] = comp_df_state_date_sector["kWh"] - comp_df_state_date_all["kWh"]\n'

## All and sector data tie - Looks like a difference in rounding, so both data are good to use

In [ ]:
#_ = comp_df_state_date_sector["difference"].plot(figsize=(20,5),title="Difference between sector sum and all")

# Electricity by Sector Plots

## Electric Account data doesn't start until 2008

In [ ]:
df_electric_population[["electric_accounts","year","sector","state","kWh"]].groupby("year").count().style.applymap(lambda x: 'background-color : red' if x==0 else 'background-color : green')

,electric_accounts,sector,state,kWh
year,,,,
2001,0,3444,3444,3444
2002,0,3444,3444,3444
2003,0,3444,3444,3444
2004,0,3444,3444,3444
2005,0,3444,3444,3444
2006,0,3444,3444,3444
2007,0,3444,3444,3444
2008,2820,3444,3444,3444
2009,2816,3444,3444,3444


## By Sector

I'm not sure if these are displaying what you think they are. The values are the same in each plot. For example, look at August 2015 for AR. (Miles)

In [ ]:
unique_sectors = df_electric_population["sector"].unique().tolist()

sorted(unique_sectors)

['ALL', 'COM', 'IND', 'OTH', 'RES', 'TRA']

In [ ]:
for sector in unique_sectors:
    
    query = df_electric_population.query(f'(sector == "{sector}") & year > 2007 & (state=="TX" | state=="HI" | state=="CA")').sort_values(["date","sector"],ascending=True)

    query_mean = (df_electric_population.query(f'(sector == "{sector}") & year > 2007').sort_values(["date","sector"],ascending=True)
                .groupby(["date"])).mean()


    vis = (px.scatter(query,x="date",y="kWh_percapita",facet_col="sector",facet_col_wrap=2,color="state"))

    vis.update_yaxes(matches=None)

    vis.add_trace(go.Scatter(x=query_mean.index, y=query_mean["kWh_percapita"],
                        mode='lines',
                        name='USA Average'))


    fn = f"{sector}_kWh_percapita.html"
    vis.write_html(f"data/electricity_retail_sales/visuals/{fn}")

    vis.show()

# US Populuation PEr capita

In [ ]:
"""
unique_state = df_electricity_sales_sector_clean_population["state"].unique().tolist()
fig = make_subplots(rows=len(unique_sectors), cols=1)
i = 1
for index,state in enumerate(unique_state):
    query = df_electricity_sales_sector_clean_population.query(f'sector == "{sector}"')

    x = query["year"].to_list()
    y = query["kWh_percapita"].to_list()

    fig.append_trace(
        go.Scatter(x=x, y=y,mode='markers',marker_color='rgba(152, 152, 152, .8)',name=query["state"].unique().tolist(),),
        row=i, col=1
    )
    i += 1

fig.update_layout(height=800, width=800, title_text="Side By Side Subplots")
fig.show()

"""

# Merge US Population Data with Retail sales

# Read in US Census Data

In [ ]:
fn = "census_2010_2021.csv"

In [ ]:
df_total_electric_sales_clean.merge

# Per Capita Calculation

In [ ]:
#Calculate kWh per capita
df_electric_population["kWh_percapita"] = df_electric_population["kWh"] / df_electric_population["electric_accounts"]

In [ ]:
for sector in unique_sectors:
    
    query = df_electric_population.query(f'(sector == "{sector}") & year > 2007 & (state=="TX" | state=="HI" | state=="CA")').sort_values(["date","sector"],ascending=True)

    query_mean = (df_electric_population.query(f'(sector == "{sector}") & year > 2007').sort_values(["date","sector"],ascending=True)
                .groupby(["date"])).mean()


    vis = (px.scatter(query,x="date",y="kWh_percapita",facet_col="sector",facet_col_wrap=2,color="state"))

    vis.update_yaxes(matches=None)

    vis.add_trace(go.Scatter(x=query_mean.index, y=query_mean["kWh_percapita"],
                        mode='lines',
                        name='USA Average'))


    fn = f"{sector}_kWh_percapita.html"
    vis.write_html(f"data/electricity_retail_sales/visuals/{fn}")

    vis.show()